In [36]:
# Importing libraries for data analysis
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sea

# to ignore stupid warnings
import warnings
warnings.filterwarnings("ignore")

# importing library for webs Scraping
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [44]:
# Creating an array of all the students roll numbers including lateral enteries
lords_normal_students = np.arange(160920733001, 160920733180)
lords_lateral_entry = np.arange(160920733301, 160920733320)
mj_normal_students = np.arange(160420733001, 160420733122)
mj_lateral_entry = np.arange(160420733301, 160420733313)
deccan_normal_students = np.arange(160320733001, 160320733091)
deccan_lateral_entry = np.arange(160320733301, 160320733310)

# Combining both the arrays into one
total_students = np.concatenate((lords_normal_students, lords_lateral_entry, mj_normal_students, mj_lateral_entry, deccan_normal_students, deccan_lateral_entry), axis=0)

# Removing roll numbers of students that discontionued and whose roll nunbers cannot be found on the webite
total_students = total_students[~((total_students == 160920733156) | (total_students == 160320733078) | (total_students == 160920733176))]

In [45]:
# Creating a dictionary that maps the subject code to subject
subject_dic = {
    "502N" : "Computer Networks Lab",
    "502O" : "Artificial Intelligence Lab",
    "574" : "Software Engineering Lab",
    "585": "Artificial Intelligence",
    "503I": "Object Oriented Analysis Design",
    "518" : "Automata Language Computation",
    "535" : "Software Engineering",
    "538" : "Computer Networks",
    "503F" : "Principals of Programming Language",
    "596"  : "BLOCK CHAIN TECHNOLOGIES",
    "503J" : "DATA SCIENCE"
}


In [46]:
# Creating an empty dataframe whose columns are the subject names of the semester  

results_df = pd.DataFrame(index=total_students)
results_df.index.name = "Roll Number"
results_df

""
Roll Number
160920733001
160920733002
160920733003
160920733004
160920733005
...
160320733305
160320733306
160320733307


In [47]:
# setting up selenium to webscrape the result from the osmania website
driver = webdriver.Chrome()
wait = WebDriverWait(driver, 10)

driver.get("https://www.osmania.ac.in/res07/20230580.jsp")
search_bar = wait.until(EC.element_to_be_clickable((By.NAME, "htno")))

for roll_number in results_df.index:
    
    search_bar.send_keys(roll_number)
    search_bar.send_keys(Keys.ENTER)
    
    search_bar = wait.until(EC.element_to_be_clickable((By.NAME, "htno")))

    rows = driver.find_elements(By.XPATH, '//*[@id="AutoNumber4"]/tbody/tr')[2:]
    name = driver.find_element(By.XPATH, '//*[@id="AutoNumber3"]/tbody/tr[3]/td[2]/b/font').text  
    gpa_list = driver.find_elements(By.XPATH, '//*[@id="AutoNumber5"]/tbody/tr')
    
    current_sem_gpa = gpa_list[-1].find_element(By.XPATH, 'td[2]/b/font').text
    
    results_df.loc[roll_number, ["Name"]] = name
    credits = 0
    backs = 0
    pending = 0
    
    for row in rows:
        sub_code = row.find_element(By.XPATH, 'td[1]/b/font').text.strip()
        grade = row.find_element(By.XPATH, 'td[5]/b/font').text.strip()
        credit = float(row.find_element(By.XPATH, 'td[3]/b/font').text.strip())
        
        if sub_code in subject_dic.keys():
            subject = subject_dic[sub_code]
            results_df.loc[roll_number, [subject]] = grade
            
            if grade == "F" or grade == 'Ab':
                backs +=1
            else:
                credits += credit
                
        else:
            if grade == 'F' or grade == 'Ab':
                backs += 1
                pending += credit
                
    results_df.loc[roll_number, ["Current Sem Credits"]] = credits
    results_df.loc[roll_number, ["Backlogs"]] = backs
    results_df.loc[roll_number, ["Previous Pending"]] = pending
    results_df.loc[roll_number, ["Overall GPA"]] = current_sem_gpa
    
driver.quit()
results_df.head(10)

,Name,Computer Networks Lab,Artificial Intelligence Lab,Principals of Programming Language,Artificial Intelligence,Automata Language Computation,Software Engineering,Computer Networks,Software Engineering Lab,Object Oriented Analysis Design,Current Sem Credits,Backlogs,Previous Pending,Overall GPA,DATA SCIENCE,BLOCK CHAIN TECHNOLOGIES
Roll Number,,,,,,,,,,,,,,,,
160920733001,MOHAMMED MUZAMMIL ALI,S,A,C,D,C,C,B,A,D,21.0,1.0,3.0,PASSED-7.19,NaN,NaN
160920733002,MOHD MOIZ UDDIN,S,S,C,A,C,C,A,A,C,21.0,0.0,0.0,PASSED-7.95,NaN,NaN
160920733003,ATEEQ BIN MUBARAK WAHLAN,A,S,B,B,C,B,A,A,B,21.0,0.0,0.0,PASSED-8.19,NaN,NaN
160920733004,WASIF ALI KHAN,S,S,B,B,B,A,A,S,B,21.0,0.0,0.0,PASSED-8.57,NaN,NaN
160920733005,SYED HUMAYUN RASHEED,S,S,B,S,D,C,A,S,C,21.0,0.0,0.0,PASSED-8.14,NaN,NaN
160920733006,MOHD SUFYAAN KHAN,A,S,C,B,D,A,S,A,C,21.0,0.0,0.0,PASSED-8.05,NaN,NaN
160920733007,MAAZ MOHAMMED HAQQANI,S,A,D,D,D,D,B,B,D,21.0,0.0,0.0,PASSED-6.71,NaN,NaN
160920733008,MIR YOUSUF ALI BUQARI,A,A,D,A,C,B,C,B,D,21.0,0.0,0.0,PASSED-7.38,NaN,NaN
160920733009,HANZALA JAVED ALI SHAIK,S,S,B,B,C,A,B,S,B,21.0,0.0,0.0,PASSED-8.29,NaN,NaN


In [50]:
#Creating a column called Total Credits that contains all the credits that a student has upto current Sem
results_df["Total Credits"] = 83 - results_df["Previous Pending"] + results_df["Current Sem Credits"]

In [53]:
# saving the results df in a csv file locally
results_df.to_csv('results.csv')